In [63]:
import json
from matplotlib import pyplot as plt
import pandas as pd
import os
import glob
import numpy as np
from empiricaldist import Cdf
import shutil
pd.set_option('display.max_colwidth', None)

add_path = ''
is_test = 0

base_path = '../simulator/bin/simData'
if add_path: base_path += f'/{add_path}'

metric_to_measure = [
    'pbusy',
    'ptransmit',
    'pqu',
    'preset',
    'pstable',
    'pfailed',
    'psent',
    'redundancy_constant',
    't',
    'interval',
    'Nnbr',
    'counter',
    'reward'
]
metric_to_measure_slotframe = [*metric_to_measure, 'eb_prob', "eb_nbr", "eb_mbr"]

metric_to_measure_conv = [['reward', 'psent', 'pbusy', 'pqu'],['Nnbr', 'redundancy_constant', 't', 'preset', 'ptransmit']]

# Rank Result
!cd /Users/dzakyzf/lain/iotlab/simulator/bin; python3.7 compute_kpis.py --path={add_path}

13969.74s - pydevd: Sending message related to process being replaced timed-out after 5 seconds


compute_kpis.py:723: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  "(", "").str.replace(")", "").str.split(", ").str[0].astype(int)
compute_kpis.py:725: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  "(", "").str.replace(")", "").str.split(", ").str[1].astype(int)
simData/exp4_metrics


In [46]:
df = pd.read_csv(f'{base_path}/comparison_merged.csv', sep='\t')
# df = df[df.columns[~df.columns.str.contains('-std|dead')]]
df.sort_values(by=['rpljoin_time_m-mean-mean'], inplace=True, ignore_index=True, ascending=False)
if not is_test: sel_algo = df['ori_name'].head(1).values[0]
df.style.highlight_min(color = 'lightcoral', axis = 0).highlight_max(color = 'crimson', axis = 0)

,method,trickle_reward-mean-mean,trickle_reward-mean-std,rpljoin_time_m-mean-mean,rpljoin_time_m-mean-std,current-consumed-mean-mean,current-consumed-mean-std,last_pfailed--mean,last_pfailed--std,last_psent--mean,last_psent--std,trickle_pbusy-mean-mean,trickle_pbusy-mean-std,dead_nodes-mean,dead_nodes-std,alive_nodes-mean,alive_nodes-std
0,qt_exp4_addrem1_1,0.855690,0.022117,12.638169,0.302088,0.104230,0.005396,0.041164,0.004113,0.958836,0.004113,0.356739,0.004107,0.000000,0.000000,49.000000,0.000000
1,qt_exp4_addrem2_1,0.809461,0.041793,10.434033,0.812112,0.103385,0.004405,0.051458,0.009046,0.948542,0.009046,0.366140,0.010286,0.000000,0.000000,49.000000,0.000000


# Convergence

In [47]:
# define the name of the directory to be deleted
fig_path = f"{base_path}/figures"

def remove_create_folder(path):
    try:
        shutil.rmtree(path)
    except OSError as e:
        pass

    try:
        os.makedirs(path)
    except OSError as e:
        pass

remove_create_folder(fig_path)

### Convergence chart

In [49]:
from sklearn.preprocessing import minmax_scale

sel_algo = 'qt_exp4_addrem2_1'
conv_base_path_figures = os.path.join(base_path, 'conv', sel_algo, 'figures')
remove_create_folder(conv_base_path_figures)

conv_base_path_data = os.path.join(base_path, 'conv', sel_algo, 'data')
remove_create_folder(conv_base_path_data)


for infile in glob.glob(os.path.join(base_path, sel_algo, '*.dat.json')):
    with open(infile, 'r') as f:
        data = json.load(f)
        
        for run_id in data.keys():
            if int(run_id) > 0: continue
            for mote_id in data[run_id].keys():
                if mote_id == 'global-stats':
                    continue
                
                for mid, metrics in enumerate(metric_to_measure_conv):
                    result = {x:[] for x in metrics}
                    for it in data[run_id][mote_id]['trickle']:
                        trickle_it = int(it)
                        trickle_data = data[run_id][mote_id]['trickle'][it]

                        for k in metrics:
                            val = trickle_data[k]
                            if val is not None:
                                result[k].append(val)
                            else:
                                result[k].append(0)

                    conv_df = pd.DataFrame(result)
                    fig = plt.figure(figsize=(15,5), dpi=300)
                    for k in result.keys():
                        res = result[k]
                        if max(res) > 1:
                            res = minmax_scale(res)
                        plt.plot(res, label=k, linestyle='--', marker='o')

                    it_len = len(data[run_id][mote_id]['trickle'])
                    plt.legend()
                    plt.title(f'Trickle of {sel_algo}')
                    plt.xlabel('Trickle iteration')
                    plt.ylabel('Values')
                    plt.tight_layout()
                    plt.savefig(f"{conv_base_path_figures}/mid{mid}_it{it_len}_mote{mote_id}_1.png")
                    plt.close()
                    conv_df.to_excel(f"{conv_base_path_data}/mid{mid}_it{it_len}_mote{mote_id}_1.xlsx", index=False)

In [116]:
raise "efefe"

TypeError: exceptions must derive from BaseException

## Prev

In [138]:
subfolders = list(
    [os.path.join(base_path, x) for x in os.listdir(base_path)]
)

main_result = {}

for subfolder in subfolders:
    if not os.path.isdir(subfolder):
        continue
    
    mthd = subfolder.split('/')[-1]
    for infile in glob.glob(os.path.join(subfolder, '*.dat.json')):
        with open(infile, 'r') as f:
            data = json.load(f)

            result = {}
            for k in metric_to_measure:
                new_key = f'trickle_{k}'
                result[new_key] = {}

            for k in metric_to_measure_slotframe:
                new_key = f'slotframe_{k}'
                result[new_key] = {}

            per_slotframe_run = 0
            for run_id in data.keys():
                for mote_id in data[run_id].keys():
                    if mote_id == 'global-stats':
                        per_slotframe_run = data[run_id]['global-stats']['per_slotframe_run'] 
                        result['per_slotframe_run'] = per_slotframe_run
                        continue
                
                    for it in data[run_id][mote_id]['trickle']:
                        trickle_it = int(it)
                        trickle_data = data[run_id][mote_id]['trickle'][it]

                        for k in metric_to_measure:
                            new_key = f'trickle_{k}'
                            if trickle_it not in result[new_key].keys():
                                result[new_key][trickle_it] = []
                            
                            if k in trickle_data.keys():
                                new_key = f'trickle_{k}'
                                val = trickle_data[k]
                                if val is not None:
                                    result[new_key][trickle_it].append(val)

                    for k in metric_to_measure_slotframe:
                        new_key = f'slotframe_{k}'
                        values = data[run_id][mote_id].get(f'per_slotframe_{k}', [])

                        for idx, val in enumerate(values):
                            if idx not in result[new_key].keys():
                                result[new_key][idx] = []
                            result[new_key][idx].append(val)
            algo = infile.split("/")[-2]
            main_result[algo] = result

### Figure per algorithm

In [63]:
conv_base_path = f'{fig_path}/Figure per algorithm/'
remove_create_folder(conv_base_path)

# Trickle states
for algo in sorted(main_result.keys()):
    fig = plt.figure(figsize=(15,5), dpi=300)
    
    for k in metric_to_measure:
        metric = f'trickle_{k}'

        data = []
        is_over = False
        for state in sorted(main_result[algo][metric].keys()):
            values = main_result[algo][metric][state]
            if not values or all(i is None for i in values): continue
            values = [x for x in values if x is not None]
            if len(values) == 0: continue
            mean_ = np.mean(values)
            if np.max(values) > 1: is_over = True
            data.append(mean_)
        
        if not is_over: plt.plot(data, label=metric, linestyle='--', marker='o')
        
    plt.legend()
    plt.title(f'Trickle of {algo}')
    plt.xlabel('Trickle states')
    plt.ylabel('Values')
    plt.tight_layout()
    plt.savefig(f"{conv_base_path}/Trickle_{algo}.png")
    plt.close()

# Per slotframe
for algo in sorted(main_result.keys()):
    fig = plt.figure(figsize=(15,5), dpi=300)
    per_slotframe_run = main_result[algo]['per_slotframe_run']
    
    for k in metric_to_measure_slotframe:
        metric = f'slotframe_{k}'

        data = []
        is_over = False
        for slotframe in sorted(main_result[algo][metric].keys()):
            values = main_result[algo][metric][slotframe]
            if not values or all(i is None for i in values): continue
            values = [x for x in values if x is not None]
            if len(values) == 0: continue
            if isinstance(values[0], dict): continue
            mean_ = np.mean(values)
            if np.max(values) > 1: is_over = True
            data.append(mean_)
        
        if not is_over:
            range_ = [x*per_slotframe_run for x in range(1, len(data)+1)]
            plt.plot(range_, data, label=metric, linestyle='--', marker='o')
    
    plt.legend()
    plt.title(f'Slotframe of {algo}')
    plt.xlabel('Slotframe')
    plt.ylabel('Values')
    plt.tight_layout()
    plt.savefig(f"{conv_base_path}/Slotframe_{algo}.png")
    plt.close()

### Figure per metric

In [45]:
conv_base_path = f'{fig_path}/Figure per metric/'
remove_create_folder(conv_base_path)

# Trickle states
for k in metric_to_measure:
    fig = plt.figure(figsize=(15,5), dpi=300)
    metric = f'trickle_{k}'

    for algo in sorted(main_result.keys()):

        data = []
        for state in sorted(main_result[algo][metric].keys()):
            values = main_result[algo][metric][state]
            if not values or all(i is None for i in values): continue
            values = [x for x in values if x is not None]
            if len(values) == 0: continue
            mean_ = np.mean(values)
            data.append(mean_)
        
        plt.plot(data, label=f'{metric}_{algo}', linestyle='--', marker='o')
        
    plt.legend()
    plt.title(f'Trickle of {metric}')
    plt.xlabel('Trickle states')
    plt.ylabel('Values')
    plt.tight_layout()
    plt.savefig(f"{conv_base_path}/metric_{metric}.png")
    plt.close()

# Per slotframe
for k in metric_to_measure_slotframe:
    fig = plt.figure(figsize=(15,5), dpi=300)
    metric = f'slotframe_{k}'
    
    for algo in sorted(main_result.keys()):
        data = []
        per_slotframe_run = main_result[algo]['per_slotframe_run']
        for slotframe in sorted(main_result[algo][metric].keys()):
            values = main_result[algo][metric][slotframe]
            if not values or all(i is None for i in values): continue
            values = [x for x in values if x is not None]
            if len(values) == 0: continue
            if isinstance(values[0], dict): continue
            mean_ = np.mean(values)
            data.append(mean_)
        
        range_ = [x*per_slotframe_run for x in range(1, len(data)+1)]
        plt.plot(range_, data, label=f'{metric}_{algo}', linestyle='--', marker='o')
    
    plt.legend()
    plt.title(f'Slotframe of {metric}')
    plt.xlabel('Slotframe')
    plt.ylabel('Values')
    plt.tight_layout()
    plt.savefig(f"{conv_base_path}/metric_{metric}.png")
    plt.close()

In [46]:
# algo = 'ori_5_exp3_random3_runs1_motes10_minutes60_disprio1_autoeb3_autok1_autot1_ql1'
# metric = 'slotframe_nbr'


# data = []
# for slotframe in sorted(main_result[algo][metric].keys()):
#     values = main_result[algo][metric][slotframe]
#     mean_ = int(np.ceil(np.mean(values)))
#     data.append(mean_)

In [47]:
# algo = 'ori_2_exp3_random3_runs1_motes10_minutes60_disprio1_autoeb3_autok1_autot1_ql1'
# metric = 'trickle_pbusy'

# data = []
# for state in sorted(main_result[algo][metric].keys()):
#     values = main_result[algo][metric][state]
#     mean_ = np.mean(values)
#     data.append(mean_)

In [48]:
# temp = np.array(data) + data[-1]
# aa = np.ceil(np.arange(5, 2, -0.1))
# data2 = [*data, *aa, *temp]
# # 
# eb_prob = 1/4
# max_ = 1 - eb_prob

In [49]:
# total = 0
# avgs = []
# avg = 0
# prev_val = None
# move = []
# move2 = []

# for idx, val in enumerate(data2):
#     total += val
#     avg = total / (idx+1)
#     avgs.append(avg)

#     m = 1
#     move.append(m)
#     eb_used_prob = eb_prob
#     eb_used_prob += (max_ / pow(2, val))
    
#     move2.append(eb_used_prob)
#     prev_val = val

# plt.plot(move2)
# plt.plot(len(move2) * [eb_prob])
# plt.show()
# plt.close()

In [50]:
# plt.plot(data2)
# plt.plot(avgs)
# plt.plot(move)
# plt.show()
# plt.close()

### CDF all algorithms

In [139]:
conv_base_path = f'{fig_path}/CDF all algorithms'
remove_create_folder(conv_base_path)

# Trickle states
for k in metric_to_measure:
    fig = plt.figure(figsize=(10,10), dpi=300)
    metric = f'trickle_{k}'

    for algo in sorted(main_result.keys()):
        data = []
        for state in sorted(main_result[algo][metric].keys()):
            values = main_result[algo][metric][state]
            if not values or all(i is None for i in values): continue
            values = [x for x in values if x is not None]
            if len(values) == 0: continue
            mean_ = np.mean(values)
            data.append(mean_)
        
        if len(data) > 0: Cdf.from_seq(data).plot()
        
    plt.legend(main_result.keys())
    plt.title(metric)
    plt.xlabel('Values')
    plt.ylabel('CDF')
    plt.tight_layout()
    plt.savefig(f"{conv_base_path}/{metric}_cdf.png")
    plt.close()


# Per slotframe
for k in metric_to_measure_slotframe:
    fig = plt.figure(figsize=(10,10), dpi=300)
    metric = f'slotframe_{k}'
    
    for algo in sorted(main_result.keys()):
        data = []
        per_slotframe_run = main_result[algo]['per_slotframe_run']
        for slotframe in sorted(main_result[algo][metric].keys()):
            values = main_result[algo][metric][slotframe]
            if not values or all(i is None for i in values): continue
            values = [x for x in values if x is not None]
            if len(values) == 0: continue
            if isinstance(values[0], dict): continue
            mean_ = np.mean(values)
            data.append(mean_)
        
        if len(data) > 0: Cdf.from_seq(data).plot()
    
    plt.legend(main_result.keys())
    plt.title(metric)
    plt.xlabel('Values')
    plt.ylabel('CDF')
    plt.tight_layout()
    plt.savefig(f"{conv_base_path}/{metric}_cdf.png")
    plt.close()